<a href="https://colab.research.google.com/github/pythonuzgit/elmurodov/blob/master/Image_classification_using_CNN_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"uzazam","key":"7abb2778b77e603b3a87069f481fc6a6"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#change the permission

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d dibakarsil/9-classes-noisy-image-dataset

100% 5.34G/5.35G [01:54<00:00, 29.4MB/s]
100% 5.35G/5.35G [01:54<00:00, 50.1MB/s]


In [4]:
from zipfile import ZipFile
file_name = "/content/9-classes-noisy-image-dataset.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

Checking for device

In [7]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
print(dev)

cuda


Transforms

In [9]:
transformer = transforms.Compose([
                                  transforms.Resize((150, 150)),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.ToTensor(),
                                  transforms.Normalize([0.5, 0.5, 0.5],
                                                       [0.5, 0.5, 0.5])
])

Load training and test datasets

In [11]:
train_path = '/content/dataset/train_im'
test_path = '/content/dataset/test_im'


train_loader = DataLoader(
    torchvision.datasets.ImageFolder(
        train_path, transform = transformer), batch_size = 256, shuffle = True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(
        test_path, transform = transformer), batch_size = 256, shuffle = True
)

Categories

In [12]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1]
                  for j in root.iterdir()])
print(classes)

['erlang', 'exponential', 'gaussian', 'lognormal', 'poisson', 'rayleigh', 'saltpepper', 'speckle', 'uniform']


In [19]:
len(classes)

9

Build the Model

In [28]:
class ConvNet(nn.Module):
  def __init__(self, num_classes = 6):
    super(ConvNet, self).__init__()

    #Output size after convolutional filter
    #((w - f + 2*P)/ s) + 1

    #input shape = (256, 3, 150, 150)

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3,
                           stride = 1, padding = 1)
    
    #Shape = (256, 12, 150, 150)

    self.bn1 = nn.BatchNorm2d(num_features = 12)

    #Shape = (256, 12, 150, 150)

    self.relu1 = nn.ReLU()

    #Shape = (256, 12, 150, 150)

    self.pool = nn.MaxPool2d(kernel_size = 2)

    #Reduce the image size be factor 2
    #Shape = (256, 12, 75, 75)

    self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3,
                           stride = 1, padding = 1)
    
    #Shape = (256, 20, 150, 150)

    self.relu2 = nn.ReLU()

    #Shape = (256, 20, 75, 75)

    self.conv3 = nn.Conv2d(in_channels = 20, out_channels = 32,
                           kernel_size = 3, stride = 1, padding = 1)

    #Shape = (256, 32, 75, 75)

    self.bn3 = nn.BatchNorm2d(num_features = 32)

    #Shape = (256, 32, 75, 75)

    self.relu3 = nn.ReLU()

    #Shape = (256, 32, 75, 75)

    self.fc = nn.Linear(in_features = 32*75*75, out_features = num_classes)

  def forward(self, input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)

    output = self.pool(output)

    output = self.conv2(output)
    output = self.relu2(output)

    output = self.conv3(output)
    output = self.bn3(output)
    output = self.relu3(output)

    output = output.view(-1, 32 * 75 * 75)

    output = self.fc(output)
    return output

Defining training parameters

In [29]:
model = ConvNet(num_classes = 9)
model = model.cuda()


Loss and optimizer

In [35]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [36]:
num_epochs = 25

Calculating training and testing datasets

In [37]:
train_count = len(glob.glob(train_path + '/**/*.jpg'))
test_count = len(glob.glob(test_path + '/**/*.jpg'))

In [38]:
print(train_count, test_count)

12000 2000


In [41]:
best_accuracy = 0.0

for epoch in range(num_epochs):
  model.train()
  train_accuracy = 0.0
  train_loss = 0.0

  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available:
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())

    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.cpu().data * images.size(0)
    _, prediction = torch.max(outputs.data, 1)
    train_accuracy += int(torch.sum(prediction == labels.data))

  train_accuracy = train_accuracy/train_count
  train_loss = train_loss/train_count




  model.eval()

  test_accuracy = 0.0

  for i, (images, labels) in enumerate(test_loader):
    if torch.cuda.is_available:
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())

    outputs = model(images)
    _, prediction = torch.max(outputs.data, 1)
    test_accuracy += int(torch.sum(prediction == labels.data))

 
  test_accuracy = test_accuracy/test_count



  print('Epoch: '+str(epoch)+' Train loss: '+str(int(train_loss))+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))


Epoch: 0 Train loss: 0 Train Accuracy: 0.76025 Test Accuracy: 0.22
Epoch: 1 Train loss: 0 Train Accuracy: 0.7798333333333334 Test Accuracy: 0.2955
Epoch: 2 Train loss: 0 Train Accuracy: 0.7935 Test Accuracy: 0.2355
Epoch: 3 Train loss: 0 Train Accuracy: 0.8101666666666667 Test Accuracy: 0.2945
Epoch: 4 Train loss: 0 Train Accuracy: 0.8105833333333333 Test Accuracy: 0.2275
Epoch: 5 Train loss: 0 Train Accuracy: 0.8278333333333333 Test Accuracy: 0.2145
Epoch: 6 Train loss: 0 Train Accuracy: 0.8225833333333333 Test Accuracy: 0.2895
Epoch: 7 Train loss: 0 Train Accuracy: 0.8273333333333334 Test Accuracy: 0.34
Epoch: 8 Train loss: 0 Train Accuracy: 0.8055833333333333 Test Accuracy: 0.2885
Epoch: 9 Train loss: 0 Train Accuracy: 0.761 Test Accuracy: 0.227
Epoch: 10 Train loss: 0 Train Accuracy: 0.80075 Test Accuracy: 0.222
Epoch: 11 Train loss: 0 Train Accuracy: 0.8158333333333333 Test Accuracy: 0.242
Epoch: 12 Train loss: 0 Train Accuracy: 0.8365833333333333 Test Accuracy: 0.35
Epoch: 13 Tra